In [1]:
from docx import Document
from pdfminer.high_level import extract_pages
from docx2python import docx2python
import re
from copy import copy
import unicodedata
path = r"C:\ecoit_summer\data\database\2006_EVNHANOIHGC-KTVH\Du thao QCKT AI camera v3 2024.docx"

# Test Read


In [2]:

def clean_text(text):
    """
    Remove all hyperlinks from the input text, keeping only the visible text.
    Supports formats like <a href="...">text</a> and raw URLs.
    """
    # Remove HTML anchor tags but keep the text inside
    text = re.sub(r'<a [^>]*>(.*?)</a>', r'\1', text, flags=re.IGNORECASE)
    # Remove raw URLs (http, https, www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = unicodedata.normalize("NFKD", text)
    return text.strip()

def extract_text_blocks(doc_path):
    doc = docx2python(doc_path)
    print(dir(doc))
    all_text = doc.body  # This returns a nested list of text by section → paragraph → run
    flat_text = []
    tables = []
    i = 0
    for section in all_text:
          # Debugging line to check structure
        if len(section) > 1:
                print("Table detected")
                tables.append(copy(section))
                # print(section)
                flat_text.append(f"Table - {i}\n")
                i += 1
                continue
        
        for para in section:
            for line in para:
                for run in line:
                    if run.strip():
                        flat_text.append(clean_text(run.strip()))
    return tables, flat_text

# def is_bullet(text):
#     return text.startswith(("•", "-", "*", "▪", "‣", "–"))

# def is_numbered(text):
#     return re.match(r"^\(?[0-9]+[\.\)]", text) or re.match(r"^[IVXLC]+\.", text)

def text_depth(text):
    text = text.strip()
    if re.match(r'^(?=[IVXLCDM]+\s*[\.\)])[IVXLCDM]+[\.\)]+[\t\s]', text):
        return 0
    elif re.match(r"^\(?[0-9]+[\.\)]+[\t\s]", text):
        return 1
    elif re.match(r'^[a-z]+[\.\)/]+[\t]', text):
        return 2
    elif re.match(r"^[--]+[\t]", text):
        return 3
    elif text.startswith("+"):
        return 4
    else:
        return -1

def extract_lists_with_docx2python(doc_path):
    tables, items = extract_text_blocks(doc_path)
    print(len(tables), len(items))  # Debugging line to check lengths
    levels = [text_depth(i) for i in items]
    print(len(levels), len(items))  # Debugging line to check lengths
    return tables, (levels, items)
# Run it
tables, (levels, items) = extract_lists_with_docx2python(path)
print(items)
for level, item in zip(levels, items):
    print(f"Level: {level}, Item: {item}")

# print("🔸 Bullet List Items:")
# for item in bullets:
#     print("-", item)

# print("\n🔢 Numbered List Items:")
# for item in numbers:
#     print(item)


['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_pars', 'body', 'body_pars', 'body_runs', 'close', 'comments', 'core_properties', 'document', 'document_pars', 'document_runs', 'docx_reader', 'endnotes', 'endnotes_pars', 'endnotes_runs', 'footer', 'footer_pars', 'footer_runs', 'footnotes', 'footnotes_pars', 'footnotes_runs', 'header', 'header_pars', 'header_runs', 'html_map', 'image_folder', 'images', 'officeDocument', 'officeDocument_pars', 'officeDocument_runs', 'properties', 'save_images', 'text']
Table detected
Table detected
2 47
47 47
['TẬP ĐOÀN ĐIỆN LỰC VIỆT NAM', 'T

In [3]:
def parse_table(section):
    table = []
    for line in section: 
        fields = []
        for cell in line:
            res = []
            for par in cell: 
                for run in par.runs:
                    if run.text != '':
                        res.append(run.text)
        
            fields.append(clean_text(" ".join(res)))
        table.append(fields)
    print(table)
    return table

def ranking_table(table): 
    def cell_depth(cell):
            cell = cell.strip()

            patterns = [
                (r'^[IVXLCDM]\.?.*', 1),          # I, II, etc. with optional dot
                (r'^\d+(\.\d+)*\.?.*', 2),                         # 1, 1.1, 1.2.3 with optional ending dot
                (r'^[a-z][\.\)/]?.*', 3),                       # a, a), b., C), etc.
                (r'^[-+]*', 4),                                   # -, +
            ]

            for pattern, depth in patterns:
                if re.match(pattern, cell):
                    return depth
            return -1  # No match — likely body text

    ranked_table = list()
    for line in table: 
        pos = -1
        level = -1
        depth = []
        for index, cell in enumerate(line): 
            level = cell_depth(cell)
            if level > -1:
                depth = cell.split(".")
                print(level, cell)
                pos = index
                break
        ranked_table.append([[[pos, level], depth], line[1:]])
    return ranked_table


    

In [4]:
def cell_depth(cell):
        cell = cell.strip()

        patterns = [
            (r'^[IVXLCDM]\.?.*', 1),          # I, II, etc. with optional dot
            (r'^\d+(\.\d+)*\.?.*', 2),                         # 1, 1.1, 1.2.3 with optional ending dot
            (r'^[a-z][\.\)/]', 3),                       # a, a), b., C), etc.
            (r'^[-+]?.*', 4),                                   # -, +
        ]

        for pattern, depth in patterns:
            if re.match(pattern, cell):
                return depth
        return -1  # No match — likely body text
print(cell_depth('12.1'))

2


In [5]:
def text_depth(text):
    text = text.strip(" ")
    gap = len(text)
    text = text.lstrip("\t")
    gap -= len(text)
    org = [-1 for i in range(gap)]
    if re.match(r'^(?=[IVXLCDM]+\s*[\.\)])[IVXLCDM]+[\.\)]+[\t\s]', text):
        org.append(0)
    elif re.match(r"^\(?[0-9]+[\.\)]+[\t\s]", text):
        org.append(1)
    elif re.match(r'^[a-z]+[\.\)/]+[\t\s]', text):
        org.append(2)
    elif re.match(r"^[-]+[\t\s]", text):
        org.append(3)
    elif text.startswith("+"):
        org.append(4)
    else:
        org.append(-1)
    return org

In [6]:
doc = docx2python(path, duplicate_merged_cells=False)
all_pars = doc.body_pars
tables = []
for section in all_pars:
    print("Len section:", len(section))
    if len(section) > 1:
        print("Table found")
        tables.append(ranking_table(parse_table(section)))
    else: 
        for para in section:
            print("Len para:", len(para))
            for pars in para:
                print("Len pars:", len(pars))
                for par in pars:
                    runs = [run for run in par.runs if len(run.text) > 0]
                    if len(runs) == 0:
                        continue
                    print(par.list_position, text_depth(str(runs[0].text)), runs)

Len section: 1
Len para: 1
Len pars: 3
(None, []) [-1] [Run(html_style=[], text='TẬP ĐOÀN ĐIỆN LỰC VIỆT NAM')]
(None, []) [-1] [Run(html_style=[], text='TỔNG CÔNG TY')]
(None, []) [-1] [Run(html_style=[], text='ĐIỆN LỰC TP HÀ NỘI')]
Len section: 1
Len para: 1
Len pars: 1
(None, []) [-1] [Run(html_style=[], text='DỰ THẢO')]
Len section: 1
Len para: 1
Len pars: 1
(None, []) [-1] [Run(html_style=[], text='DỰ THẢO')]
Len section: 1
Len para: 2
Len pars: 1
Len pars: 4
(None, []) [-1] [Run(html_style=[], text='CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM')]
(None, []) [-1] [Run(html_style=[], text='Độc lập – Tự do – Hạnh phúc')]
Len section: 1
Len para: 1
Len pars: 19
(None, []) [-1] [Run(html_style=[], text='QUY CÁCH KỸ THUẬT')]
(None, []) [-1] [Run(html_style=[], text='Hệ thống thiết bị camera AI giám sát lưới điện cao thế 220kV/110kV')]
(None, []) [-1] [Run(html_style=[], text='(Ban hành kèm theo Quyết định số:              /QĐ-EVNHANOI ngày    /      /2024)')]
('18', [1]) [1] [Run(html_style=[], t

Document format \\
Prior - prior...: statement

In [7]:
def parse_content(title, content):
    return [f"({t}) {c}" for t, c in zip(title, content) if len(c) > 0]

def roman_to_int(roman):
    roman_values = {
        'I': 1, 'V': 5, 'X': 10,
        'L': 50, 'C': 100,
        'D': 500, 'M': 1000
    }

    total = 0
    prev_value = 0

    for char in reversed(roman.upper()):
        value = roman_values.get(char, 0)
        if value < prev_value:
            total -= value
        else:
            total += value
            prev_value = value

    return total

def character_to_int(char):
    return ord(char) - ord('a') + 1

def numel_to_int(str):
    return int(str)

def to_int(str, order=0):
    match order:
        case 1:
            return roman_to_int(str)
        case 2: 
            return numel_to_int(str)
        case 3: 
            return character_to_int(str)


In [ ]:
formatted_tables = []
title_line_idx = 0
title_row = None
separator = "-"
heads = dict()
output_contents = []
for table in tables:
    father = None
    table_idx = len(output_contents)
    print(table_idx)
    for index, line in enumerate(table):
        if index == title_line_idx:
            title_row = line[1]
            continue

        (pos, level), content = line[0], line[1]
        numbering = pos[1]
        level_int = [to_int(entry, order = numbering) for entry in level]
        print(line)
        if numbering - 1 in heads.keys():
            father = heads[numbering - 1]
        else:
            father = None
            heads[numbering] = index
        # Overwriting to current index
        
        if len(level_int) > 1: 
            father = heads[numbering]
        else:
            heads[numbering] = index
        output_str = separator.join(tuple(parse_content(title_row, content))).strip("-").replace("--", "-")
        prior, path = separator.join(content).strip("-").replace("--", "-"), []
        if father is not None:
            father_node = output_contents[father]
            output_str = father_node[1] + ": " + output_str
            prior = father_node[1] + "/" + prior
            path = father_node[0] + [level_int[-1]]
        print(index, father, (numbering, level_int), (path, prior, output_str))
        output_contents.append((path, prior, output_str))
    formatted_tables.append(output_contents[table_idx:])
        
        

0
[[[0, 2], ['1']], ['CNTT', 'Công nghệ thông tin.']]
1 None (2, [1]) ([], 'CNTT-Công nghệ thông tin.', '(Từ viết tắt) CNTT-(Nghĩa đầy đủ) Công nghệ thông tin.')
[[[0, 2], ['2']], ['CSDL', 'Cơ sở dữ liệu.']]
2 None (2, [2]) ([], 'CSDL-Cơ sở dữ liệu.', '(Từ viết tắt) CSDL-(Nghĩa đầy đủ) Cơ sở dữ liệu.')
[[[0, 2], ['3']], ['ADSS', 'All Dielectric Self Supporting Optical Cable - Cáp quang tự treo cách điện hoàn toàn.']]
3 None (2, [3]) ([], 'ADSS-All Dielectric Self Supporting Optical Cable - Cáp quang tự treo cách điện hoàn toàn.', '(Từ viết tắt) ADSS-(Nghĩa đầy đủ) All Dielectric Self Supporting Optical Cable - Cáp quang tự treo cách điện hoàn toàn.')
[[[0, 2], ['4']], ['IP', 'Internet Protocol - giao thức liên mạng (giao thức Internet).']]
4 None (2, [4]) ([], 'IP-Internet Protocol - giao thức liên mạng (giao thức Internet).', '(Từ viết tắt) IP-(Nghĩa đầy đủ) Internet Protocol -

In [15]:
output_contents

[([],
  'CNTT-Công nghệ thông tin.',
  '(Từ viết tắt) CNTT-(Nghĩa đầy đủ) Công nghệ thông tin.'),
 ([],
  'CSDL-Cơ sở dữ liệu.',
  '(Từ viết tắt) CSDL-(Nghĩa đầy đủ) Cơ sở dữ liệu.'),
 ([],
  'ADSS-All Dielectric Self Supporting Optical Cable - Cáp quang tự treo cách điện hoàn toàn.',
  '(Từ viết tắt) ADSS-(Nghĩa đầy đủ) All Dielectric Self Supporting Optical Cable - Cáp quang tự treo cách điện hoàn toàn.'),
 ([],
  'IP-Internet Protocol - giao thức liên mạng (giao thức Internet).',
  '(Từ viết tắt) IP-(Nghĩa đầy đủ) Internet Protocol - giao thức liên mạng (giao thức Internet).'),
 ([],
  'LAN-Local Area Network - Mạng cục bộ.',
  '(Từ viết tắt) LAN-(Nghĩa đầy đủ) Local Area Network - Mạng cục bộ.'),
 ([],
  'NMTĐ-Nhà máy Thủy điện.',
  '(Từ viết tắt) NMTĐ-(Nghĩa đầy đủ) Nhà máy Thủy điện.'),
 ([],
  'VT-Viễn thông.',
  '(Từ viết tắt) VT-(Nghĩa đầy

# DOCX 


In [10]:
import docx
from docx import Document

doc = Document(path)

# Collect all tables as lists of lists
all_tables = []
for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)
    all_tables.append(table_data)
# print(len(all_tables))
# print(type(all_tables[0]))
# Collect all non-table text (paragraphs not inside tables)
i = 0
valid = True
non_table_text = []
for para in doc.paragraphs:
    # print(f"Checking paragraph {i}: {para._element.getparent().tag}")
    if para._element.getparent().tag.endswith('tbl'):
        print(f"Checked paragraph {i} as it is inside a table")
        if valid is True: 
            valid = False
            non_table_text.append(all_tables.pop(0))
        # This paragraph is inside a table
    else:
        valid = True
    if para.text.strip():  
        non_table_text.append(para.text.strip())

# for t in non_table_text:
#     if isinstance(t, list):
#         print("Table detected...")
#         for row in t:
#             print(" | ".join(row))
#         print("-"*40)
#         continue
#     print(t)


AttributeError: 'list' object has no attribute 'seek'

# Docs read

In [ ]:
from docx import Document
from docx.oxml.ns import qn
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
from copy import copy

doc = Document(path)
def is_list_paragraph(paragraph):
    """
    Returns True if the paragraph is part of a numbered or bulleted list.
    """
    p = paragraph._p  # XML element for paragraph
    numPr = p.find(qn('w:numPr'))
    return numPr is not None


def get_list_level(paragraph):
    """
    Returns the nesting level of the list item (0 = top level), or None if not in list.
    """
    p = paragraph._p
    pPr = p.find(qn('w:pPr'))
    if pPr is not None:
        numPr = pPr.find(qn('w:numPr'))
    else:
        return None
    if numPr is not None:
        ilvl = numPr.find(qn('w:ilvl'))
        level = numPr.find(qn('w:numId'))
        if level is not None or ilvl is not None:
            return int(ilvl.get(qn('w:val'))), int(level.get(qn('w:val')))
    return None

def get_list_indicator(num_id, level, count):
    """Approximate indicator: a., b., 1., etc. — only works for flat lists"""
    if level == 0:
        return f"{count}."
    elif level == 1:
        return f"{chr(96 + count)}."  # a. b. c.
    elif level == 2:
        return f"{roman(count)}."  # i. ii. iii.
    return f"{count}."


def iter_block_items(parent):
    """
    Yield paragraphs and tables in the order they appear in the document.
    """
    for child in parent.element.body.iterchildren():
        if child.tag == qn('w:p'):
            yield Paragraph(child, parent)
        elif child.tag == qn('w:tbl'):
            yield Table(child, parent)

# Now iterate and detect table positions
tables_found = 0
for i, block in enumerate(iter_block_items(doc)):
    if isinstance(block, Table):
        tables_found += 1
        print(f"Table {tables_found} found at position {i}")
    elif isinstance(block, Paragraph):
        if i == 9:
            item = copy(block)
        print(f"Paragraph at position {i}, {get_list_level(block)}: {block.text.strip()}")

Paragraph at position 0, None: TỔNG CÔNG TY	CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM
Paragraph at position 1, None: ĐIỆN LỰC TP HÀ NỘI	Độc lập - Tự do - Hạnh phúc
Paragraph at position 2, None: BAN KỸ THUẬT
Paragraph at position 3, None: Số: 07/KT	Hà Nội, ngày 07 tháng 01 năm 2025
Paragraph at position 4, None: TỜ TRÌNH
Paragraph at position 5, None: Về việc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV)
Paragraph at position 6, None: Kính gửi: Ông Phó Tổng Giám đốc Kỹ thuật
Paragraph at position 7, None: Căn cứ vào Quyết định số 8594/QĐ-EVNHANOI ngày 06/12/2023 Tổng Công ty Điện lực TP Hà Nội về việc giao danh mục mua sắm tài sản năm 2024 đợt 2 cho Công ty Lưới điện Cao thế thành phố Hà Nội;
Paragraph at position 8, None: Ban Kỹ thuật kính trình Tổng Giám đốc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái công nghiệp (UAV) để tổ chức thực hiện mua sắm bổ sung trang bị cho Công ty Lưới điện cao thế TP Hà Nội với các nội dung sau:
Paragraph at position 9, (0, 1):

In [ ]:
from docx import Document
from docx.oxml.ns import qn

doc = Document(path)

# Track counter for each numId + ilvl
list_counters = {}

def get_list_info(p):
    pPr = p._p.find(qn('w:pPr'))
    numPr = pPr.find(qn('w:numPr')) if pPr is not None else None
    if numPr is not None:
        numId = numPr.find(qn('w:numId'))
        ilvl = numPr.find(qn('w:ilvl'))
        if numId is not None and ilvl is not None:
            return int(numId.get(qn('w:val'))), int(ilvl.get(qn('w:val')))
    return None, None

def get_list_indicator(num_id, level, count):
    """Approximate indicator: a., b., 1., etc. — only works for flat lists"""
    if level == 0:
        if level == 0 and num_id == 6:
            return f"{chr(96 + count)}."
        return f"{count}."
    elif level == 1:
        if num_id == 6:
            return f".{count}"
        return f"{chr(96 + count)}."  # a. b. c.
    elif level == 2:
        return f"{roman(count)}."  # i. ii. iii.
    return f"{count}."

def roman(num):
    """Convert integer to lowercase roman numeral"""
    val = [
        (10, 'x'), (9, 'ix'), (5, 'v'),
        (4, 'iv'), (1, 'i')
    ]
    result = ''
    for (n, roman_char) in val:
        while num >= n:
            result += roman_char
            num -= n
    return result

for para in doc.paragraphs:
    text = para.text.strip()
    numId, ilvl = get_list_info(para)
    if numId is not None:
        key = (numId, ilvl)
        list_counters[key] = list_counters.get(key, 0) + 1
        count = list_counters[key]
        marker = get_list_indicator(numId, ilvl, count)
        print(f"{numId}/{ilvl}: {marker} {text}")
    else:
        print(text)


TỔNG CÔNG TY	CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM
ĐIỆN LỰC TP HÀ NỘI	Độc lập - Tự do - Hạnh phúc
BAN KỸ THUẬT
Số: 07/KT	Hà Nội, ngày 07 tháng 01 năm 2025
TỜ TRÌNH
Về việc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV)
Kính gửi: Ông Phó Tổng Giám đốc Kỹ thuật
Căn cứ vào Quyết định số 8594/QĐ-EVNHANOI ngày 06/12/2023 Tổng Công ty Điện lực TP Hà Nội về việc giao danh mục mua sắm tài sản năm 2024 đợt 2 cho Công ty Lưới điện Cao thế thành phố Hà Nội;
Ban Kỹ thuật kính trình Tổng Giám đốc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái công nghiệp (UAV) để tổ chức thực hiện mua sắm bổ sung trang bị cho Công ty Lưới điện cao thế TP Hà Nội với các nội dung sau:
1/0: 1. Các yêu cầu chung về nhà sản xuất
2/0: 1. Nhà sản xuất phải có chứng nhận quản lý chất lượng ISO hoặc tương đương.
2/0: 2. Nhà thầu phải cung cấp đầy đủ thông tin của sản phẩm trong phần thuyết minh đáp ứng yêu cầu kỹ thuật và bảng phụ lục đặc tính kỹ thuật cam kết (của tập tiêu chuẩn kỹ thuật này), đồ

In [ ]:
import re

def clear_hyperlinks(text):
    """
    Remove all hyperlinks from the input text, keeping only the visible text.
    Supports formats like <a href="...">text</a> and raw URLs.
    """
    # Remove HTML anchor tags but keep the text inside
    text = re.sub(r'<a [^>]*>(.*?)</a>', r'\1', text, flags=re.IGNORECASE)
    # Remove raw URLs (http, https, www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    return text.strip()

# Example usage:
sample = 'Visit <a href="https://example.com">our site</a> or http://test.com for more info.'
print(clear_hyperlinks(sample))